In [28]:
import pandas as pd 
import numpy as np

subset_method = "Union"
correlation_type = "Pearson"
organ_1 = "*instances01"
organ_2 = "*instances1.5x"

correlations_mean = pd.read_excel(f"../../data/page6_LinearXWASCorrelations/average_correlations/Correlations_comparisons_{subset_method}_{correlation_type}.xlsx", index_col=[0, 1]).loc[(organ_1, organ_2)]

correlations_std = pd.read_excel(f"../../data/page6_LinearXWASCorrelations/average_correlations/Correlations_sd_comparisons_{subset_method}_{correlation_type}.xlsx", index_col=[0, 1]).loc[(organ_1, organ_2)]

KeyError: '*instances1.5x'

In [29]:
pd.read_excel(f"../../data/page6_LinearXWASCorrelations/average_correlations/Correlations_comparisons_{subset_method}_{correlation_type}.xlsx", index_col=[0, 1]).loc[(organ_1)]

,Phenotypic,Genetics,Alcohol,AnthropometryBodySize,AnthropometryImpedance,ArterialStiffness,BloodBiochemistry,BloodCount,BloodPressure,BoneDensitometryOfHeel,...,medical_diagnoses_W,medical_diagnoses_X,medical_diagnoses_Y,medical_diagnoses_Z,All,All_Biomarkers,All_Environmental,All_Socioeconomics,All_Phenotypes,All_Diseases
organ_2,,,,,,,,,,,,,,,,,,,,,
Average,0.362023,NaN,0.732327,0.585667,0.334504,0.341576,0.601253,0.321278,0.224072,0.735782,...,0.136286,0.143736,0.132006,0.75949,0.285531,0.271499,0.738981,0.706273,0.598984,0.498838
*,0.419783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
from plotly.graph_objs import Figure, Bar

correlations_mean.sort_values(ascending=False, inplace=True)
correlations_std = correlations_std[correlations_mean.index]
fig = Figure()
fig.add_trace(
    Bar(
        x=correlations_mean.index,
        y=correlations_mean,
        error_y={"array": correlations_std, "type": "data"},
        name="Average correlations",
        marker_color="indianred",
    )
)
fig.update_layout(xaxis_tickangle=-90)
fig.update_layout({"width": 1800, "height": 600})

OtherSociodemographics              0.150558
GeneralHealth                       0.169207
Education                           0.303801
Claudication                        0.186431
Smoking                             0.272199
                                      ...   
medical_diagnoses_B                 0.365124
CognitiveReactionTime               1.005038
HandGripStrength                    1.005038
CognitiveSymbolDigitSubstitution    0.970265
medical_diagnoses_X                 0.582895
Name: (MainDimensions, Average), Length: 91, dtype: float64